<a href="https://colab.research.google.com/github/HzcIrving/Deep-Learning-for-myself/blob/master/RNN_learning1_textgeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  使用RNN来生成文本
- **工具** -- 使用character-based RNN.  
- **数据集** -- 使用Shakersperare's writing
  - 给定一段莎士比亚风格的字符串，训练模型，反复调用模型来自动生成这种风格的诗词。
  - 给GPU使能来获得更快的执行速度：
    -  In Colab: Runtime > Change runtime type > Hardware acclerator > GPU.
- 这个教程保护可执行代码实现（通过调用keras [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). 下面这个是输出的样本，当教程训练30个epoch，然后以“Q”为开始输出的话剧风格：

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

* 这个模型是基于字符串的，当训练开始时，这个模型不知道如何拼写英语单词。
* 输出的结构类似于文本的播放块，通常以说话者名称开头，使用与数据集类似的所有大写字母。
*如下图所示，该模型针对小批文本(每批100个字符)进行训练，仍然能够生成具有连贯结构的较长的文本序列

## 1. 配置

### 1.1 导入库

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf 
tf.enable_eager_execution()
"""
TensorFlow 的 Eager Execution 是一种命令式编程环境，
可立即评估操作，无需构建图：操作会返回具体的值，而
不是构建以后再运行的计算图。这样能让您轻松地开始使
用 TensorFlow 和调试模型，并且还减少了样板代码。
"""

import numpy as np 
import os 
import time 

## 1.2 下载莎士比亚数据集

In [0]:
path_to_file = tf.keras.utils.get_file(
  'shakespeare.txt',
  'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

1122304/1115394 [==============================] - 0s 0us/step


## 1.3  阅读数据

In [0]:
text = open(path_to_file,'rb').read().decode(encoding='utf-8')
# 文本长度，表示是文本中包含字符串的总数量
print('文本长度:{} 字符'.format(len(text)))

# 看一下头250个字符
print(text[:250])

# 文件中的独特字符
vocab = sorted(set(text))
print('{} 特殊字符'.format(len(vocab)))
print(vocab)

文本长度:1115394 字符
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

65 特殊字符
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## 1.4 处理文本
### 1.4.1 对文本进行向量化
在训练之前，我们需要将字符串映射为字符表示。我们需要生成两个查字表，一个将字符映射到数字，另一个将数字映射到字符。

In [0]:
# 将特殊字符映射到索引
char2idx = {u: i for i,u in enumerate(vocab)} # i:index,u:char
print(char2idx)
idx2char = np.array(vocab)
print(idx2char)

text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int) # 输出text中每个字符对应的index
print(text_as_int.shape)
print(text_as_int[1:10])
print(text[1:10])

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
[18 47 56 ... 45  8  0]
(1115394,)
[47 56 57 58  1 15 47 58 47]
irst Citi


本教程：如何使用基于字符的RNN生成文本：
- Andrej Karpathy 在 The Unreasonable Effectiveness of Recurrent Neural Networks 一文中提供的莎士比亚作品数据集；
- 虽然有些句子合乎语法规则，但大多数句子都没有意义。该模型尚未学习单词的含义。
  -  该模型是基于字符的模型。在训练之初，该模型都不知道如何拼写英语单词，甚至不知道单词是一种文本单位。
  - 输出的文本结构仿照了剧本的结构：文本块通常以讲话者的名字开头，并且像数据集中一样，这些名字全部采用大写字母。
  - 如下文所示，尽管该模型只使用小批次的文本（每批文本包含 100 个字符）训练而成，但它仍然能够生成具有连贯结构的更长文本序列。

In [2]:
# 库
import tensorflow as tf 
tf.enable_eager_execution()

import numpy as np 
import os 
import time 

# 下载数据集
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1122304/1115394 [==============================] - 0s 0us/step


In [0]:
# 读取数据
# 看一些文本内容
text = open(path_to_file).read()
# text的长度，是里面所有字符数
print('text字符长度数: {}'.format(len(text)))

# 看一下头1000个字符
print(text[:1000])


In [0]:
# 处理文本
# 1.向量化
# 将字符串映射到数字表示值。
"""创建两个对照表：
一个用于将字符映射到数字，
另一个用于将数字映射到字符。"""

# 查看特殊字符'
vocab = sorted(set(text))
print('{}种字符'.format(len(vocab)))

# 创建一个特殊字符到index的映射
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
# print(char2idx)
# 每一个字符都有一个对应的整数表示值，从0到len(vocab)的索引映射

text_as_int = np.array([char2idx[c] for c in text]) 
# print(text_as_int)

for char,_ in zip(char2idx,range(20)):
  print('{:6s}-->{:4d}'.format(repr(char),char2idx[char]))
  
# 显示文本的前13个字符如何映射到整数
print('{} ---- characters mapped to int ---->{}'.format(text[:13],text_as_int[:13]))


#预测任务
根据给定的字符或字符序列预测下一个字符最有可能是什么？这是我们要训练模型去执行的任务。模型的输入将是字符序列，而我们要训练模型去预测输出，即每一个时间步的下一个字符。<br>
由于 RNN 会依赖之前看到的元素来维持内部状态，那么根据目前为止已计算过的所有字符，下一个字符是什么？

In [0]:
# 创建训练样本和目标
# 将文本化为训练样本和训练目标。
"""每个训练样本都包含从文本中选取的 seq_length 个字符。
相应的目标也包含相同长度的文本，但是将所选的字符序列
向右顺移一个字符。例如，假设 seq_length 为 4，我们的
文本为“Hello”，则可以将“Hell”创建为训练样本，将
“ello”创建为目标。"""

# 将文本拆分为文本块，每个块长度为seq_length+1
# 我们希望单个字符输入的最大长度句
seq_length = 100

# 建立训练样本以及训练目标
chunks = tf.data.Dataset.from_tensor_slices(text_as_int).batch(seq_length+1,\
                                            drop_remainder = True)

for item in chunks.take(5):
  print(repr(''.join(idx2char[item.numpy()]))) # repr

# 利用此文本块创建输入文本和目标文本
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text,target_text 

# 将文本块映射到上面的函数
dataset = chunks.map(split_input_target)
         
# 输出第一个样本前10个值
print('\n'*2)
for input_example,target_example in dataset.take(1):
  print('训练样本:',repr(''.join(idx2char[input_example.numpy()])))
  print('训练目标:',repr(''.join(idx2char[target_example.numpy()])))
print('\n'*2)

# 这些向量每个索引均作为一个时间步来处理
# 时间步0，输入：映射到18的字符，并尝试预测映射到47的字符；
# 时间步1，相同操作，除了当前字符外，还要考虑上一步信息
for i,(input_idx,target_idx) in enumerate(zip(input_example[:5],target_example[:5])):
  print("时间步：{:4d}".format(i))
  print("输入：{}({:s})".format(input_idx,repr(idx2char[input_idx])))
  print("期望输出：{}({:s})".format(target_idx,repr(idx2char[target_idx])))
print('\n'*2)


In [0]:
# 使用tf.data创建批次文本并重排这些批次
"""使用tf.data将文本分块，在将这些数据馈送到模型中之前，需要对数据进行
重排，并将其打包成批"""

# Batch size 
BATCH_SIZE  = 64

# BUFFERSIZE来shuffle数据集
# TF数据被设计用来处理可能无限的序列,所以它不会试图在内存中打乱整个序列
# 实际上，其维护一个缓冲区，在其中对元素进行shuffle
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

# 模型
  # 嵌入层：一个可训练的对照表，它会将每个字符的数字映射到具有 embedding_dim 个维度的高维度向量；
  # GRU层：一种层大小等于单位数的 RNN。（在此示例中，您也可以使用 LSTM 层。）
  # 密集层：带有vocab_size个单元
class Model(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,units):
    super(Model,self).__init__()
    self.units = units 
    
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    
    if tf.test.is_gpu_available():
      self.gru = tf.keras.layers.CuDNNGRU(self.units,
                                        return_sequences=True,
                                        recurrent_initializer='glorot_uniform',
                                        stateful=True)
    else:
      self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   recurrent_initializer='glorot_unifrom',
                                   stateful=True)
      
    self.fc = tf.keras.layers.Dense(vocab_size)      
  
  def call(self,x):
    embedding = self.embedding(x)
    
    # 每个时间步输出
    # 输出shape = (batch_size,seq_length,hidden_size)
    output = self.gru(embedding)
    
    # dense layer会输出每个时间步（seq_length）的预测值
    # 在denselayer之后的输出shape = (seq_length*batch_size,vocab_size)
    prediction = self.fc(output)
    
    # 在训练过程中，状态将被用于通过模型的每一步
    return prediction 

# 实例化模型、优化器与Loss function
# vocabulary字符长度
vocab_size = len(vocab)

# embedding 维度
embedding_dim = 256

# RNN单元数量
units = 1024

model = Model(vocab_size,embedding_dim,units)

# 优化器 adam 
optimizer = tf.train.AdamOptimizer()

# 使用系数交叉熵,不需要建立one-hot
def loss_function(real,preds):
  return tf.losses.sparse_softmax_cross_entropy(labels=real,logits=preds)

model.build(tf.Tensorshape([BATCH_SIZE,seq_length]))
model.summary()

In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))
print('\n'*2)

vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
print('\n'*2)

# 对于输入字符串的期望最大序列长度
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 建立训练样本与训练目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])
print('\n'*2)
  
# 批处理方法允许我们轻松地将这些单个字符转换为所需大小的序列。
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))
print('\n'*2)  
  
# 定义划分函数，并映射到数据库
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
print('\n'*2)

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))
print('\n'*2)

# Batch Size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch // BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)
# dataset

# # 构建模型
# 嵌入层：一个可训练的对照表，它会将每个字符的数字映射到具有 embedding_dim 个维度的高维度向量；
# GRU 层：一种层大小等于单位数的 RNN。（在此示例中，您也可以使用 LSTM 层。）
# 密集层：带有 vocab_size 个单元。
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools 
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid'
  )

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

# 检查输出维度
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
print('\n'*2)

model.summary()

# 训练模型
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(
  optimizer = tf.train.AdamOptimizer(),
   loss = loss
)

# -------------配置检查点--------------------------------------
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
# -------------------------------------------------------------

EPOCHS = 3
history = model.fit(
      dataset.repeat(),
      epochs = EPOCHS,
      steps_per_epoch = steps_per_epoch,
      callbacks=[checkpoint_callback]
)

Length of text: 1115394 characters



{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}



F
i
r
s
t



'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'



Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
T

In [19]:
# 产生文本
# 重新读取最近的检查点
# 模型仅接受固定大小的批次数据。
# 要使用相同的权重和不同的模型，我们需要重建模型，并从检查点恢复权重。
tf.train.latest_checkpoint(checkpoint_dir)

"""
由于RNN状态从一个时间步传递到另一个时间步的方式，
模型只接受构建后的固定批大小。

要运行具有不同批大小的模型，
我们需要重新构建模型并从检查点恢复权重。
"""
model = build_model(vocab_size,embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))
model.summary()

def generate_text(model,start_string):
  # 产生的字符数
  num_generate = 1000
  
  # 将开始字符串转换为数字(向量化)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval,0)
  
  # 空列表，用来存储结果
  text_generated = []
  
  # temperature越低>>预测越准
  # temperature越高>>结果越出人意料
  temperature = 1.0
  
  # 这里batch_size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # 移除batch维度
    predictions = tf.squeeze(predictions,0)
    # 然后，使用多项分布计算预测字符的索引
    predictions = predictions/temperature
    predicted_id = tf.multinomial(predictions,num_samples=1)[-1,0].numpy()
    
    # 将这个作为输入
    input_eval = tf.expand_dims([predicted_id],0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model,start_string =u"ROMEO:"))
# 提升表现：
  # 1.EPOCHS提高
  # 2.不同的start_string、temperature
  # 3.改变网络结构

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_10 (CuDNNGRU)      (1, None, 1024)           3938304   
_________________________________________________________________
dense_10 (Dense)             (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
ROMEO:
Canminar, for Rome, I have; strick'd, but his not your were,
To gentle not, I have king deap of her,
As would to this neviur; neeling for thy same, four och his bears, and made us will yourselve's dispartion.
First, of lords in throunct is cannot wasces be with sus
The imingl'd velaunter:
For beion! we speak upain to you con of any leftlemen?
I be a fullent and br

###在此示例中，我们使用采用 GradientTape 的自定义训练循环。要详细了解此方法，请参阅 Eager Execution 指南。

- 首先，用零和形状（批次大小，RNN 单元数）初始化模型的隐藏状态。为此，我们将调用在创建模型时定义的函数。

- 然后，逐批对数据集进行迭代，并计算与该输入关联的预测和隐藏状态。

- 在训练过程中，发生了许多有趣的现象：

  - 模型获得隐藏状态（初始化为 0，我们称之为 H0）和第一批输入文本（我们称之为 I0）。
  - 然后，模型返回预测值 P1 和 H1。
  - 对于下一批输入，模型收到 I1 和 H1。
  - 现在，有趣的是我们将 H1 随 I1 一起传递给模型，模型正是通过这种方式进行学习。从各个批次中学习到的上下文将包含到隐藏状态中。
  - 重复上述操作，直到数据集中的数据全部用尽。然后开始一个新的周期，并重复此过程。
  - 计算预测值后，使用上面定义的损失函数计算损失。然后，计算相对于模型变量的损失梯度。

- 最后，使用 apply_gradients 函数在优化器的帮助下朝着训练的方向迈进一步。
![训练过程](https://github.com/mari-linhares/docs/blob/patch-1/site/en/tutorials/sequences/images/text_generation_training.png?raw=true)

### 使用模型生成文本
使用我们训练的模型生成文本<br>
下面的代码块可生成文本：

- 首先选择一个起始字符串，初始化隐藏状态，并设置要生成的字符数。

- 使用起始字符串和隐藏状态获取预测值。

- 然后，使用多项分布计算预测字符的索引 - 将此预测字符用作模型的下一个输入。

- 模型返回的隐藏状态被馈送回模型中，使模型现在拥有更多上下文，而不是仅有一个单词。在模型预测下一个单词之后，经过修改的隐藏状态再次被馈送回模型中，模型从先前预测的单词获取更多上下文，从而通过这种方式进行学习。

![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

-----------------------------------------------------------------------------
- repr() --> 返回对象的规范字符串表示形式。
- list[:-1] --> 保存列表中除最后一个以外的所有值
- list[1:] --> 保存列表中除第一个以外的所有值
- batch(batch_size,drop_remainder=False)
  -  drop_remainder:表示在少于batch_size元素的情况下是否应删除最后一批
  -  [数据对象Dataset的详解](https://www.imooc.com/article/68648)
- from_tensor_slices（tensors）
  - 用于创建dataset，其元素是给定张量的切片的元素。


[2, 3, 4, 5]

In [0]:
! 